# Case studies for Robot Vaccine paper

In [ ]:
import os
import pandas as pd
import numpy as np
import datetime
import itertools
import matplotlib.pylab as plt
from matplotlib import rc
rc("text", usetex=True)
rc("font", family="serif")

import run_robot
import prepare_data
from importlib import reload
reload(prepare_data)
reload(run_robot)

In [ ]:
basic_prm = prepare_data.save_basic_parameters(min_level=0.8, rep=2.5, ndays=30)
subnot_factor = 11.6
cities_data = prepare_data.compute_initial_condition_evolve_and_save(basic_prm, "SP", ["SP"], 10000000, subnot_factor, 1, "data/report_covid_with_drs_07_29.csv")
cities_data

In [ ]:
# Define the basic data for the case studies

# Basic reproduction number
basic_rep = 1.8

# Simulation horizon
# A little more than a year when thevaccine should be here
ndays = 36*14

# Mean time in ICU
time_icu = 7

# Lockdown level
lock_level = 0.8

# Define basic paramters
basic_prm = prepare_data.save_basic_parameters(min_level=lock_level, rep=basic_rep, time_icu=time_icu, ndays=ndays)

# For SP only
cities_data = prepare_data.compute_initial_condition_evolve_and_save(basic_prm, "SP", ["SP"], 10000000, subnot_factor, 1, "data/covid_with_state.csv")

In [ ]:
# Create a target matrix (max infected level)
ncities, ndays = len(cities_data.index), int(basic_prm["ndays"])
target = 0.8*np.ones((ncities, ndays))
target = prepare_data.save_target(cities_data, target)

# Use a forcedif that releases the cities in the end
force_dif = np.ones((ncities, ndays))
cities_data

## Add information on the time series that estimate the need of ICUs

We are using the time series adjusted considering that the mean ICU stay is 7 days (which lead to larger ICU capacity).

In [ ]:
if basic_prm["time_icu"] == 11:
    # Time series adjusted considering the mean ICU time is 11 days
    ts_sp = np.array([0.0074335, 0.01523406, -0.00186355, 0.0, 1.67356018, -0.68192908, np.sqrt(0.00023883),
        0.007682840158843, 0.007536060983504])
    ts_notsp = np.array([0.00520255, 0.01532709, 0.00044498, 0.0, 1.75553282, -0.76360711, np.sqrt(3.567E-05),
        0.005426447471187, 0.005282217308748])
elif basic_prm["time_icu"] == 7:
    # Time series adjusted considering the mean ICU time is 7 days
    ts_sp = np.array([0.01099859, 0.02236023, 0.00370254, 0.0, 1.79119571, -0.80552926, np.sqrt(0.00034005),
        0.011644768910252, 0.011221496171591])
    ts_notsp = np.array([0.0076481, 0.0218084, 0.00367839, 0.0, 1.81361379, -0.82550856, np.sqrt(8.028E-05),
        0.007907216664912, 0.007721801045322])
else:
    raise NotImplementedError

ts_drs = ts_sp*np.ones((len(cities_data), len(ts_notsp)))
ts_drs = pd.DataFrame(data=ts_drs, index=cities_data.index, columns=[
    "rho_min", "rho_max", "intercept", "trend", "phi_1", "phi_2", "sigma_omega", "state0", "state_less_1"
])
ts_drs["confidence"] = 0.9
ts_drs["time_icu"] = time_icu
cities_data = pd.concat([cities_data, ts_drs], axis=1)
cities_data

In [ ]:
pd.set_option("display.width", 120)

# Simple function to run a test and save results
def run_a_test(basic_prm, result_file, figure_file, cities_data, M, target, force_dif, pools=None, budget=0, tests_off=np.zeros(0, int), tau=3, test_efficacy=0.8, daily_tests=0, proportional_tests=False, verbosity=1):
    run_robot.prepare_optimization(basic_prm, cities_data, M, target, hammer_data, force_dif, pools, 
        verbosity=verbosity, test_budget=budget, tests_off=tests_off, tau=tau, test_efficacy=test_efficacy, 
        daily_tests=daily_tests, proportional_tests=proportional_tests)
    run_robot.optimize_and_show_results(basic_prm, figure_file, result_file, cities_data, target, verbosity=verbosity)

In [ ]:
# Define mobility matrix.
M = prepare_data.convert_mobility_matrix_and_save(cities_data, max_neighbors=5, drs="data/report_drs_mobility.csv")
hammer_data = prepare_data.save_hammer_data(cities_data, 0, basic_prm["min_level"])
run_robot.find_feasible_hammer(basic_prm, cities_data, M, target, hammer_data, out_file=None, 
    incr_all=True, verbosity=1)
M

## Vaccination test

In [ ]:
reload(run_robot)

# TODO: Get rid of this "test" stuff
tau, test_efficacy, daily_tests, budget = 3, 0.8, 0, 0
tests_off = np.zeros(0, int)

print("******************** Running vaccine simulation")

# Case 1 Optimal tests
basic_prm["alternate"] = 0.0
result_file = f"results/vaccine_profile.csv"
figure_file = f"results/vaccine_profile.png"
run_a_test(basic_prm, result_file, figure_file, cities_data, M, target, force_dif, None, budget, tests_off, 
    tau, test_efficacy, daily_tests);

plt.close("all")


In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

solution = pd.read_csv(result_file, index_col=[0, 1, 2])

def retrieve_vaccination_profile(sol, pop=0, dose=0):
    sol = sol.loc[:, pop, :]
    seir = sol.loc["s", :] + sol.loc["e", :] + sol.loc["i", :] + sol.loc["r", :]
    applied = sol.loc["v", dose]*seir.loc[dose]
    return applied[14:]

In [ ]:
retrieve_vaccination_profile(solution, 0, 0).plot(lw=3, alpha=0.7, label="Pop 1, dose 1")
retrieve_vaccination_profile(solution, 0, 1).plot(lw=3, alpha=0.7, ls="--", label="Pop 1, dose 2")
retrieve_vaccination_profile(solution, 1, 0).plot(lw=3, alpha=0.7, label="Pop 2, dose 1")
retrieve_vaccination_profile(solution, 1, 1).plot(lw=3, alpha=0.7, ls="--", label="Pop 2, dose 2")
plt.legend()
plt.title("Two populations");

In [ ]:
total_vaccination = (
    retrieve_vaccination_profile(solution, 0, 0) + retrieve_vaccination_profile(solution, 0, 1) +    
    retrieve_vaccination_profile(solution, 1, 0) + retrieve_vaccination_profile(solution, 1, 1)
)
total_vaccination.plot(lw=3, label="Total doses")
plt.legend()
plt.title("Total daily vaccination");

In [ ]:
sum(total_vaccination)